### Cylindrical cells 

In [1]:
import numpy as np

import matplotlib.pyplot as plt

%matplotlib qt


In [2]:
### Typical number of cells around the cylinder
n_circum = 8
### Typical number of cells along z
n_length = 7
### Distance between 2 cells
l_0 = 1. 
### Cells heights
h_0 = 1.

In [3]:
rho_c = n_circum * l_0 / (2 * np.pi)
rho_lumen = rho_c - h_0

delta_theta = 2 * np.pi / n_circum
delta_z = delta_theta * rho_c * np.sqrt(3)/2.

zt_grid = np.mgrid[:n_length, :n_circum]
thetas = zt_grid[1].astype('float')
thetas[::2, ...] += 0.5
thetas *= delta_theta
thetas = thetas.flatten()
zeds = zt_grid[0].astype('float')
zeds *= delta_z
zeds -= zeds.max() / 2
zeds = zeds.flatten()

pos = np.zeros((n_circum * n_length, 3))

pos[:, 0] = np.cos(thetas) * rho_c
pos[:, 1] = np.sin(thetas) * rho_c
pos[:, 2] = zeds

In [5]:
import graph_tool.all as gt

In [6]:
l_0

1.0

In [7]:
cells_graph, pos_vp = gt.geometric_graph(pos, l_0*1.1)

In [8]:
rhos = np.linalg.norm(pos[:, :2], axis=1)
z_angle = np.pi/24
d_theta = np.pi/12

pseudo_x = pos[:, 2] * np.cos(z_angle) - rhos * np.sin(thetas + d_theta) * np.sin(z_angle)
pseudo_y = rhos * np.cos(thetas + d_theta)

pseudo_pos = gt.graph_draw(cells_graph, output='generated_cells.png')

![the generated cells](generated_cells.png)

In [9]:
import pandas as pd
pos = pd.DataFrame(pos, columns=['x', 'y', 'z'])


In [10]:
pos.index.name = 'cell'

pos.head()


,x,y,z
cell,,,
0,1.176320,0.487248,-2.598076
1,0.487248,1.176320,-2.598076
2,-0.487248,1.176320,-2.598076
3,-1.176320,0.487248,-2.598076
4,-1.176320,-0.487248,-2.598076


In [11]:
from scipy.spatial import Voronoi, voronoi_plot_2d

In [16]:
thetas.min(), thetas.max()

(-2.748893571891069, 3.1415926535897931)

In [114]:
thetas = np.arctan2(pos['y'], pos['x'])
rhos = np.hypot(pos['y'], pos['x'])
pos['theta'] = thetas
pos['sigma'] = thetas * rhos
pos['rho'] = rhos
pos['sigma_r'] = ((thetas) % (2*np.pi)) * rhos

voronoi = Voronoi(pos[['z', 'sigma']].values)
voronoi_r = Voronoi(pos[['z', 'sigma_r']].values)

#plt.plot(pos.z, pos.sigmas, 'bo')
fig, (ax, ax_r) = plt.subplots(1, 2)

voronoi_plot_2d(voronoi, ax=ax)
voronoi_plot_2d(voronoi_r, ax=ax_r)

for c in pos.index:
    ax.text(pos.loc[c, 'z'], pos.loc[c, 'sigma'], str(c))
    ax_r.text(pos.loc[c, 'z'], pos.loc[c, 'sigma_r'], str(c))

    
ax.set_aspect('equal')
ax_r.set_aspect('equal')


In [128]:
voronoi.vertices.shape, voronoi_r.vertices.shape

unique_links = [k for k in voronoi.ridge_dict.keys()]

for k in voronoi_r.ridge_dict.keys():
    if (not k in unique_links) and (not (k[1], k[0]) in unique_links):
        unique_links.append(k)



In [129]:
print(len(unique_links))

162


In [117]:
voronoi.ridge_dict[(0, 9)], voronoi_r.ridge_dict[(9, 0)]


([17, 46], [0, 6])

In [120]:
voronoi.vertices[17]

array([-2.30940108,  1.        ])

In [121]:
graph = gt.Graph(directed=True)

zeds = graph.new_vertex_property('double')
thetas = graph.new_vertex_property('double')
is_cell_vert = graph.new_vertex_property('double')
is_junction_edge = graph.new_vertex_property('double')

idx_dict = {}

for n, point in enumerate(voronoi.points):
    v = graph.add_vertex()
    zeds[v] = point[0]
    thetas[v] = point[1] / rho_c
    is_cell_vert[v] = 1
    idx_dict[n] = graph.vertex_index[v]
    
for n, point in enumerate(voronoi.vertices):
    v = graph.add_vertex()
    zeds[v] = point[0]
    thetas[v] = point[1] / rho_c
    is_cell_vert[v] = 0
    idx_dict[n] = graph.vertex_index[v]
    
for (c0, c1), (j0, j1) in voronoi.ridge_dict:
    cell0 = graph.vertex(c0)
    cell1 = graph.vertex(c1)
    
    

In [124]:
voronoi.vertices

array([[ -2.30940108e+00,   2.00000000e+00],
       [  2.30940108e+00,   2.00000000e+00],
       [ -8.66025404e-01,   4.50000000e+00],
       [ -5.77350269e-01,   2.00000000e+00],
       [ -1.15470054e+00,   2.00000000e+00],
       [  2.30940108e+00,  -1.00000000e+00],
       [ -2.22044605e-16,  -4.00000000e+00],
       [ -2.02072594e+00,  -1.50000000e+00],
       [ -2.30940108e+00,  -1.00000000e+00],
       [  8.66025404e-01,   4.50000000e+00],
       [ -2.02072594e+00,   2.50000000e+00],
       [ -2.30940108e+00,   3.00000000e+00],
       [ -2.02072594e+00,   3.50000000e+00],
       [ -1.44337567e+00,   2.50000000e+00],
       [ -1.44337567e+00,   3.50000000e+00],
       [ -1.15470054e+00,   3.00000000e+00],
       [ -2.02072594e+00,   1.50000000e+00],
       [ -2.30940108e+00,   1.00000000e+00],
       [ -1.44337567e+00,   1.50000000e+00],
       [ -1.15470054e+00,   1.00000000e+00],
       [  1.44337567e+00,  -1.50000000e+00],
       [  2.02072594e+00,  -1.50000000e+00],
       [  

In [119]:
[17, 46] in voronoi.ridge_vertices

True

In [113]:
voronoi.vertices[17], voronoi_r.vertices[11]

(array([-2.30940108,  1.        ]), array([-6.30940108,  1.        ]))

In [96]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.gca(projection='3d')

ax.scatter(voronoi_3d.vertices[:, 0], voronoi_3d.vertices[:, 1], voronoi_3d.vertices[:, 2], c='k')
ax.scatter(voronoi_3d.points[:, 0], voronoi_3d.points[:, 1], voronoi_3d.points[:, 2], c='r')

for simplex in voronoi_3d.ridge_vertices:
    simplex = np.asarray(simplex)
    if np.all(simplex >= 0):
        ax.plot(voronoi_3d.vertices[simplex, 0], 
                voronoi_3d.vertices[simplex, 1],
                voronoi_3d.vertices[simplex, 2], 'k-')
